In [1]:
import os

In [2]:
%pwd

'/home/smruti/projects/skin_disease_classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/smruti/projects/skin_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from skin_disease_classifier.constants import *
from skin_disease_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2025-03-07 10:03:30.612093: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-07 10:03:30.893545: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "dataset")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
import tensorflow as tf
import albumentations as A
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation Data Generator (No Augmentation)
        valid_datagenerator = ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Define Augmentation with Gaussian Blur (5×5 Kernel) using Albumentations
        def augment_image(image):
            transform = A.Compose([
                A.GaussianBlur(blur_limit=(5, 5), sigma_limit=(0.1, 2.0), p=0.5),  # 5×5 Kernel
                A.HorizontalFlip(p=0.5),
                A.Rotate(limit=40, p=0.5),
                A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=0.2, p=0.5),
                A.RandomBrightnessContrast(p=0.5)
            ])
            augmented = transform(image=image)['image']
            return augmented

        def preprocess_function(image):
            image = np.array(image)  # Convert to NumPy array
            image = augment_image(image)  # Apply Gaussian Blur & Other Augmentations
            return image

        # Train Data Generator (With Augmentation)
        if self.config.params_is_augmentation:
            train_datagenerator = ImageDataGenerator(
                preprocessing_function=preprocess_function,  # Custom Augmentation
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator  # No Augmentation

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-03-07 10:03:34,427: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-07 10:03:34,429: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-07 10:03:34,431: INFO: common: created directory at: artifacts]
[2025-03-07 10:03:34,433: INFO: common: created directory at: artifacts/training]


2025-03-07 10:03:34.483919: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-07 10:03:34.542464: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-07 10:03:34.542504: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-07 10:03:34.545486: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-07 10:03:34.545546: I external/local_xla/xla/stream_executor

Found 995 images belonging to 5 classes.


/home/smruti/miniconda3/envs/tf216/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Found 3989 images belonging to 5 classes.
Epoch 1/100


...

2025-03-07 10:26:01.203499: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-07 10:26:01.203558: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2025-03-07 10:26:01.203568: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14449247232328263649
2025-03-07 10:26:01.203592: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8040045290876161522
2025-03-07 10:26:01.280437: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-07 10:26:01.280504: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: O

249/249 ━━━━━━━━━━━━━━━━━━━━ 27s 109ms/step - accuracy: 0.4080 - loss: 1.3078 - val_accuracy: 0.7742 - val_loss: 0.6382
Epoch 98/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 125us/step - accuracy: 0.3750 - loss: 1.3908 - val_accuracy: 0.6667 - val_loss: 0.8780
Epoch 99/100


2025-03-07 10:26:28.534526: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-07 10:26:28.534586: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2025-03-07 10:26:28.534597: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14449247232328263649
2025-03-07 10:26:28.534622: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8040045290876161522
2025-03-07 10:26:28.611924: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-07 10:26:28.611975: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: O

249/249 ━━━━━━━━━━━━━━━━━━━━ 27s 110ms/step - accuracy: 0.4272 - loss: 1.2917 - val_accuracy: 0.7601 - val_loss: 0.6496
Epoch 100/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 116us/step - accuracy: 0.2500 - loss: 1.5549 - val_accuracy: 0.6667 - val_loss: 0.6957


2025-03-07 10:26:55.997587: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-07 10:26:55.997870: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2025-03-07 10:26:56.073830: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-07 10:26:56.073906: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2025-03-07 10:26:56.073917: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14449247232328263649
2025-03-07 10:26:56.073963: I tensorflow/core/framework/local_re